# Tp THP Réalisé par ABOUD Ibrahim G02

**Ce Tp a pour but de implementer l'algorithm de Thomson, pour pouvoir passer d'une expression reguliere, on peux passer vers un Automate**

On peut subdiviser le probleme en 3 petits problems :
1. Implementer une structure de l'automate et toute operations entre les language des Automate (union, concat, iteration)
2. Implementer un program qui evalue l'expression reguliere et deduit les operations a faire sur les automates "l'ordre"
3. Implementer un program qui affiche l'automate de facon graphique

# 1. Structure de l'automate

On peut creer une classe "GeneralisedAutomata" qui contient les attribut suivants :
- X : l'alphabet " un tableau de character "
- S : l'ensemble d'etat " un tableau d'entier " - chaque entier est unique a un etat
- S0 : etat initiale " entier "
- F : ensemble des etats finaux " un tableau d'entier "
- I : ensemble d'instruction " un tableau de triplet (*numero_etat_precedent : entier*,*lettre : char*,*numero_etat_suivant : entier*) "

En se basant sur cette structure, on peut aussi implementer les methodes de cette classe :
- setters and getters " pour pouvoir manipuler chaque attribut "
- les additioneurs " qui imitent l'union entre deux ensemble " : *exemple : A.addF(F') <=> F = F U F'*
- les operateurs sur les automates : *exemple : A.union(A') <=> A=A'' tel que  L(A'') = L(A) U L(A')*

On vas implementer tous ce qu'on a dis jusqu'a maintenant dans ce bloc de code : " oui ce bloc de code est fait maison :) "

**Remarque :** veuillez executer le bloc de code suivant avant de passer vers le prochaine titre

**Execution d'un bloc :** cliquer sur le bloc, et apres : ctrl + Enter

In [7]:


class GeneralisedAutomata:
    # attribut de classe pour generer facilement l'ID de chaque etat -guarantir l'unicité -
    node_index = 0
    
    def __init__(self, X=[], S=[],S0=None,F=[],I=[]):
        if S==[]:
            self.S = [GeneralisedAutomata.node_index]
            self.S0 = GeneralisedAutomata.node_index
            self.F = GeneralisedAutomata.node_index
            GeneralisedAutomata.node_index+=1
        else:
            self.S=S
            self.S0=S0
            self.F=F
        self.X=X
        self.I=I
    
    def showAutomata(self):
        print("\nX: ",self.X,"\nS: ",self.S,"\nS0: ",self.S0,"\nF: ",self.F,"\nI: ",self.I)
    
    def createNodes(self,nodesNumber=1):
        nodeSet = []
        for i in range(nodesNumber):
            nodeSet.append(GeneralisedAutomata.node_index)
            GeneralisedAutomata.node_index+=1
        self.addS(nodeSet)
    
    #_________________________Setters_______________________________
    
    def setWords(self,X):
        self.X = X
    def setS0(self,S0):
        self.S0 = S0
    def setS(self,S):
        self.S=S
    def setF(self,F):
        self.F=F
    def setI(self,I):
        self.I=I
        
     #_________________________Getters_______________________________
        
    def getWords(self):
        return self.X
    def getS0(self):
        return self.S0
    def getS(self):
        return self.S
    def getF(self):
        return self.F
    def getI(self):
        return self.I
    
    #_________________________Additionneurs_______________________________
    
    def addWords(self,X):
        self.X = self.X+X
    def addS(self,S):
        self.S=self.S+S
    def addF(self,F):
        self.F=self.F+F
    def addI(self,I):
        self.I=self.I+I
    
    def removeF(self,value):
        self.F.remove(value)
    def removeI(self,value):
        self.I.remove(value)
    
    def recogniseOnly(self,letter):
        self.addS([GeneralisedAutomata.node_index])
        self.F=[GeneralisedAutomata.node_index]
        self.addI([(self.S0,letter,self.F[0])])
        GeneralisedAutomata.node_index+=1
        self.X+=[letter]
    
    
    #_________________________Operations sur Automates_______________________________
    
    # ITERATION
    def star(self):
        start = GeneralisedAutomata.node_index
        end = GeneralisedAutomata.node_index+1
        GeneralisedAutomata.node_index+=2
        self.addS([start,end])
        for i in self.F:
            self.addI([(i,"ε",end)])
        self.addI([(start,"ε",self.S0)])
        self.addI([(start,"ε",end)])
        self.addI([(end,"ε",start)])
        self.F=[end]
        self.S0=start
        
    # UNION
    def union(self,A):
        start = GeneralisedAutomata.node_index
        end = GeneralisedAutomata.node_index+1
        GeneralisedAutomata.node_index+=2
        self.addS([start,end])
        self.addS(A.getS())
        self.addI([(start,"ε",self.S0),(start,"ε",A.S0)])
        for i in self.getF():
            self.addI([(i,"ε",end)])
        for i in A.getF():
            self.addI([(i,"ε",end)])
        self.addI(A.getI())
        self.setF([end])
        self.setS0(start)
        
        set1 = set(self.X)
        set2 = set(A.X)

        self.X = list(set1.union(set2))

    # CONCATENATION
    def concat(self,A):
        start = GeneralisedAutomata.node_index
        end = GeneralisedAutomata.node_index+1
        GeneralisedAutomata.node_index+=2
        self.addS([start,end])
        self.addS(A.getS())
        for i in self.getF():
            self.addI([(i,"ε",A.getS0())])
        for i in A.getF():
            self.addI([(i,"ε",end)])
        self.addI([(start,"ε",self.getS0())])
        self.addI(A.getI())

        
        set1 = set(self.X)
        set2 = set(A.X)

        self.X = list(set1.union(set2))
        self.setF([end])
        self.setS0(start)




    

# 2. Evaluation de l'expression regulière

Selon l'algorithm de Thomson, l'evaluation se fait de facon recursive, la finalité sera qu'on demarre d'un ensemble d'Automate triviaux "reconnaitre une lettre precise", et apres utiliser les methodes defini avant pour fusionner tous les petits automate en un Automate finale.

Donc c'est comme une evaluation d'operation entre des nombres reel.

le seul probleme a resoudre sera donc : comment arriver a reconnaitre la priorité des operations

on peux le voir a travers cet exemple : 

Expression regulière : **a U b . c***
l'ordre d'execution sera le suivant :
1. tmp = c*
2. tmp = b.tmp
3. tmp = a U tmp

donc on peut voir que la notion de priorité dans les operations : " iteration, union, concat " ne differe pas de celle de "addition, multiplication, puissance"

si seulement on pouvais implementer la meme logique d'evaluation sur les expression arithmetique ...

Heureusement, "si on fait reference au cours ALSDS" on trouve que le format "postfixé" d'une operation arthmetique est peut etre pas si intuitive au humain pour la comprendre, mais tres efficace pour etre traité par la machine.

on defini alors les deux objective : 
1. une procedure qui transform l'expression de sont format original vers sont format postfixé
2. un procedure qui traite l'expression en postfixé et fais les operations sur l'automate

### Format Original vers postfixé

-realisé avec un petit aide d'internet-, mais en gros, l'ideé est : utilisation de deux piles et parcourir l'expression
le cas delicat est pour les charactere des operations ( union : + , concat : ., iteration : * ), on les empiles dans une pile appart, mais si on trouve une operation X avec une priorité inferieur a ceux dans la pile, on depile tous les Y qui valide cette condition de la pile des operations et on les empile dans la pile resultat, jusqu'a ce la condition n'est pas verifié, dans ce cas on empile X dans la pile des operations


**Remarque :** veuillez executer le bloc de code suivant avant de passer vers le prochaine titre

**Execution d'un bloc :** cliquer sur le bloc, et apres : ctrl + Enter

In [8]:
def infix_to_postfix(infix_expression):
    precedence = {'+': 1, '.': 2, '*': 3}
    stack = []
    postfix = []
    
    for symbol in infix_expression:
        if symbol.isalnum():
            postfix.append(symbol)
        elif symbol == '(':
            stack.append(symbol)
        elif symbol == ')':
            while stack and stack[-1] != '(':
                postfix.append(stack.pop())
            stack.pop()  # Remove '(' from the stack
        else:
            # Operator
            while stack and stack[-1] != '(' and precedence.get(symbol, 0) <= precedence.get(stack[-1], 0):
                postfix.append(stack.pop())
            stack.append(symbol)

    # Pop any remaining operators from the stack
    while stack:
        postfix.append(stack.pop())

    return ''.join(postfix)



### Traitement de l'expression en format Postfixé et transformation vers un automate

-fait maison- , l'idee est simple, on parcours l'expression :
- si on trouve une lettre "par exemple - a - ", alors on genere un automate qui reconnais le mot "a" et on empile l'automate
- si on trouve une operation : 
    - iteration : depiler un automate, appliquer l'iteration et rempiler
    - concatenation : depiler deux automates, appliquer la concatenation et empiler le resultat
    - union : depiler deux automates, appliquer l'union et empiler le resultat


**Remarque :** veuillez executer le bloc de code suivant avant de passer vers le prochaine titre

**Execution d'un bloc :** cliquer sur le bloc, et apres : ctrl + Enter

In [12]:
def regxToAutomata(regx):
    
    postfixe = infix_to_postfix(regx)
    
    print(postfixe)
    
    result = None
    automataStack = []
    for char in postfixe:
        if(char =="."):#concatenation
            tmp = automataStack.pop()
            result = automataStack.pop()
            result.concat(tmp)
            automataStack.append(result)
        elif(char == "*"):#iteration
            result = automataStack.pop()
            result.star()
            automataStack.append(result)
        elif(char == "+"):#union
            tmp = automataStack.pop()
            result = automataStack.pop()
            result.union(tmp)
            automataStack.append(result)
        else:
            tmp = GeneralisedAutomata()
            tmp.recogniseOnly(char)
            automataStack.append(tmp)
    return automataStack.pop()

# 3. Tracer le graphe

la partie final necessite une biblioteque python qui peut tracer les graphes de facon visible

une biblioteque reconnu pour cela est : **Graphviz**

elle defini sont propre graph, et le trace

donc on creer un graph vide

et On prend notre Automate final :
- pour chaque etat i : ajouter un noeud au graphe "un sommet nommé i"
- pour chaque transition (i,lettre,j): ajouter un arc nommé " lettre " au graph " du sommet i vers j " 


**Remarque :** veuillez executer le bloc de code suivant avant de passer vers le prochaine titre

**Execution d'un bloc :** cliquer sur le bloc, et apres : ctrl + Enter

In [11]:
import graphviz

def drawAutomata(A,expression):
    # Create an undirected graph
    dot = graphviz.Digraph(comment=expression,graph_attr={'rankdir': 'LR'})
    dot.node('Title',label=expression,shape="rectangle",penwidth='4')
    # Add nodes with attributes (e.g., name, color)
    dot.node(str(A.getS0()), label="S"+str(A.getS0()),shape="circle",penwidth='3')
    nodes = A.getS()
    nodes.remove(A.getS0())
    nodes.remove(A.getF()[0])
    for node in nodes:
        dot.node(str(node), label="S"+str(node),shape="circle")
    dot.node(str(A.getF()[0]), label="S"+str(A.getF()[0]),shape="doublecircle")
    
    for connection in A.getI():
        dot.edge(str(connection[0]), str(connection[2]),label=connection[1])
    dot.view()
    dot.render('Automate',format='pdf', cleanup=True)





# 4. Tests

maintenant vous pouver essayer de transformer des expressions regulières vers des Automates d'etat fini

- ' + ' : union
- ' . ' : concat
- ' * ' : iteration
- ' () ' : les parentheses

In [13]:
regx="(a+b)*.a.(b.c*)*"
A= regxToAutomata(regx)
drawAutomata(A,regx)

ab+*a.bc*.*.


*Tp THP Réalisé par ABOUD Ibrahim G02*